# 27-1. 프로젝트: KoChatGPT 업그레이드 하기
## 루브릭
1. 기존 KoGPT2와 SFT 적용 모델 결과 분석했는가?  
   기존 모델의 결과물과 SFT를 적용한 모델의 결과물을 정량/정성적으로 비교/분석했다.
2. SFT 모델과 RM 모델 결과 분석을 해보았는가?  
   SFT를 적용한 모델의 결과물과 RM을 적용한 모델의 결과물을 정량/정성적으로 비교/분석했다.
3. 데이터셋 정제 / 새로운 데이터셋 / foundation model 교체 중 하나를 이용해 정량적 성능 향상을 해보았는가?  
   - 기존 데이터셋을 추가로 정제하고, generation 성능을 올리기 위한 기법(Beam search, Top-k sampling 등)을 실험해 모델 성능을 향상시켰다.
   - 새로운 데이터를 수집해 전처리를 수행하여 모델의 성능을 향상시켰다.
   - 더 적절한 학습 전략(SFT, RM, PPO)을 적용하거나 initial model을 변경해 모델의 성능을 향상시켰다.
---

KoChatGPT 소스코드를 바탕으로 다양한 모델 개선 전략을 선택해 KoChatGPT를 업그레이드해 보겠습니다.

아래 제시된 전략 중 하나를 선택하거나 여러 개를 조합하여
여러분만의 custom ChatGPT를 개발해보세요. 물론 더 창의적인 좋은 아이디어를 도입해볼 수도 있겠죠?

복수의 전략을 선택했을 때 혼자서 실험해볼 시간이 부족하다면
팀을 이뤄 분업을 해보셔도 좋습니다!
1. 우리가 지난시간 살펴본 KoChatGPT 모델에 사용한 데이터셋은 아직 완벽히 정제되지 않았습니다.
2. Human Feedback이 반영된 데이터셋을 대체하기 위해 SFT와 RM 모델에 사용할 다양한 benchmark 데이터셋도 검토해볼 수 있습니다.
3. 언어모델의 생성능력을 좌우하는 최선의 디코딩을 위한 하이퍼파라미터 서치가 필요합니다.
4. 생성된 답변에 대한 주관적인 평가를 보완할 수 있는 정량적인 메트릭은 도입하지 않았었습니다.
5. LLM Trend Note1에서 살펴본 다양한 Instruction Tuning 및 Prompting 기법들도 적용해볼만 합니다.
6. 무엇보다 foundation model로 사용한 KoGPT-2는 Emergent abilities를 기대하기엔 다소 작은 사이즈의 모델입니다.  
   더 큰 파라미터 스케일을 가진 모델을 사용해보거나,
7. 더 효율적인 연산을 수행할 수 있는 LoRA의 적용 또는 새로운 Instruction Tuning 및 reward ranking 알고리즘을 도입해볼 수도 있습니다.

어떤 걸 해야할 지 감이 잡히지 않는 분들을 위해 몇 가지 예시를 소개해드리도록 하겠습니다.

## 기존 데이터셋 추가 정제
data_kochatgpt 폴더에는 세 파일이 있습니다.
1. kochatgpt_1_SFT.jsonl : SFT를 위한 prompt와 completion 문장셋
2. kochatgpt_1_RM.jsonl : RM 학습을 위한 prompt와 세 가지 ranking 문장셋
3. kochatgpt_1_PPO.jsonl : promt 문장

각 말뭉치를 EDA하여 도메인과 문체, 길이분포, 문장의 완성도 등을 분석합니다.
언어모델의 문장생성능력은 말뭉치의 전처리 수준에 큰 영향을 받습니다.
말뭉치의 분석결과를 토대로 데이터를 정제하여 모델을 재학습시켜봅니다.
(정제후 데이터셋 크기가 줄어들지 않도록, 다양한 augmentation 기법을 활용하여 크기를 유지 내지 증량합니다.)
추가 전처리 후, 기존 인퍼런스 결과와 성능을 비교해봅니다.
(주관적인 평가와 BLEU, ROUGE 등을 활용한 정량적인 평가 결과를 비교 분석하여 제시합니다.)

## 새로운 데이터셋 추가
KoChatGPT는 human feedback이 반영된 데이터를 직접 사용하는 대신
ChatGPT API를 사용하는 대안을 선택했습니다.
LLM Trend Note1 에서 살펴보았듯이
Anthropic의 RLHF는 StackExchange 같은 온라인 상의 댓글정보를 활용하여
ranking dataset을 구축해 구현되었습니다.
우리도 비슷한 로직을 적용해볼 수 있습니다.

하나의 prompt에 대한 다양한 수준의 품질로 댓글이 달린 한국어로 된 웹사이트를 찾아봅시다.
웹크롤링 기법을 사용해 reward 점수를 차등적으로 적용해볼 수 있는
instruction dataset과 ranking dataset을 구축해봅니다.

[KorQuAD 2.0](https://korquad.github.io/) 같은 한국어 이해 benchmark를 활용해 고품질의 데이터셋을 확보하고,
KoGPT-2를 사용해 빠르게 저품질 데이터셋을 페어링해볼 수도 있습니다.
다양한 데이터 증량전략을 구사하여 기존 데이터셋에 새로 구축한 데이터셋을 추가해
모델을 재학습시키고 추론 결과를 비교해 분석하여 제시해보세요.

## foundation model 교체
현재 제공되는 LMS GPU 사양으로는 수십 billion 단위 이상의 LLM을 튜닝하기 어렵습니다.
그러나 허깅페이스에서 제공하는 큰 규모의 모델을 적은 컴퓨팅 자원으로도 사용할 수 있게 해주는
경량화, 최적화 라이브러리를 사용하면
속도는 느리지만 우리의 LMS에서도 학습 및 추론이 가능해질 수 있습니다.
(힌트 : LLM Trend Note1 노드의 마지막 스텝을 참고해보세요)

허깅페이스에서 제공되는 1.2B 사이즈의 한국어 GPT pretrain model로 [skt/ko-gpt-trinity-1.2B-v0.5](https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5) 가 있습니다.
해당 모델로 foundation model을 교체해보세요.
(단 OOM 문제를 해소하기 위해 허깅페이스에서 제공하는
다양한 training argument들을 조합하여 최상의 하이퍼파라미터를 찾아내야 합니다.)
데이터셋을 아예 바꿔 모델 선택의 폭을 늘려보는 것도 좋은 선택지입니다.

foundation model 교체에 성공했다면, generator 함수를 수정하여 모델 인퍼런스 결과를 제시해보세요.

## 참고
LLM Trend Note2 노드에서 살펴본 KoChatGPT 소스코드는
빠르게 baseline모델을 설계해 실습해보기 위해 오리지널 코드를 일부 수정한 버전입니다.
프로젝트 진행을 위해 모델을 커스터마이징할 때, 필요시 "colossalai_ChatGPT_230319" 폴더 내의 원본 스크립트들을 참고하세요.

## 라이브러리 버전 확인
Torch version:1.12.1  
Cuda version: 11.3  
transformers version: 4.28.0  
GPU 사용 가능여부: True

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## skt/kogpt2-base-v2 성능확인

In [2]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


In [3]:
# 디코딩 성능 확인
max_length=50
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?


In [4]:
# 빔 서치 디코딩 + n-gram 패널티
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아


In [5]:
# 샘플링
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
그녀는 울음을 그치질 못했다.
그녀의 두 눈에는 눈물이 고이 젖어 있었다.
"오랜만에 이렇게 뵙


In [6]:
# top_p 샘플링
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그런데 그게 무슨 소리입니까?"
"아니요, 그건 아니에요."
그녀의 대답에


- Greedy Search: 매 스텝에서 가장 높은 확률의 토큰을 선택. 텍스트 생성이 이루어지지 않고 공백이 반복됨. 
- Beam Search and n-gram penalty: 여러 개의 시퀀스를 동시에 탐색하여 최종적으로 가장 가능성이 높은 시퀀스 선택. num_beam은 빔의 개수 결정. 2-gram 반복을 피함. 모델의 특별 토큰으로 끝남. 완전하지 않은 결과 생성. 반복은 피했지만 모델의 제약이 보임.
- Sampling with temperature and top-k: 확률 분포를 샘플링. 상위 50개의 토큰 중에서 선택. 창의적이고 다양한 출력을 생성.
- Top-p Sampling: 누적 확률이 90%를 차지하는 토큰들 중에서 샘플링. 자연스러운 출력을 기대했지만 \</d> 토큰이 추가됨. 어색한 결과  

샘플링되는 결과들은 달라질 수 있음.

In [7]:
list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

for prompt in list_prompt:
    max_length=50
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)
    
    # 샘플링
    output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                                 do_sample=True, temperature=2.0, top_k=50)
    print(prompt)
    print(tokenizer.decode(output_beam[0]))
    print()

불고기용 고기 한우에요?
불고기용 고기 한우에요?"
"어느 나라 음식인지는 몰라도, 왠지 햄버거 맛에 젖어 있구나, 그나저나 요거트 맛을 보고싶었는데."
"그럴 리가 없겠

리처드 닉슨이 43대 부통령직을 수행한 년도는?
리처드 닉슨이 43대 부통령직을 수행한 년도는?
닉슨은 선거운동 기간 동안 대선후보로 나서지 않았다.
그 후로 닉슨의 정치 활동은 중단된 상태다.
닉슨과 그의 친구들은 선거자금 모금에만 열중해 있었다.
닉

시카고 오헤어 국제공항은 어디에 있어?
시카고 오헤어 국제공항은 어디에 있어? 라고 질문한 적이 있었습니다.
그래서 저는 이렇게 묻고 있습니다.
그런데 정말 궁금한 게 한 두 명 뿐이 아니라는 겁니다.
오헤어로 가는 모든 항공편은 출발 전 공항 체크인부터

오늘 미세먼지 어때?
오늘 미세먼지 어때? 네. 맞습니다.
어떻게 된 건가요? 하는 생각이 듭니다. 고거 이거 어떻게 된 겁니까?
이거 무슨 말씀이시지요?
네. 예. 저는 봤습니다.
네.



## 데이터셋 확인

In [8]:
import json
import os
data_path_1_SFT = '~/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
data_path_1_SFT = os.path.expanduser(data_path_1_SFT)
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

In [9]:
# RM 데이터셋
data_path_2_RM = '~/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
data_path_2_RM = os.path.expanduser(data_path_2_RM)
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

In [10]:
# PPO 데이터셋
data_path_3_PPO = '~/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
data_path_3_PPO = os.path.expanduser(data_path_3_PPO)
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

## Supervised Fine-Tuning

In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")
tokenizer = AutoTokenizer.from_pretrained(
    "skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [13]:
# SFT 데이터셋 클래스
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [14]:
# prompt 딕셔너리 템플릿 클래스 정의
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [15]:
# 훈련셋, data collator 인스턴스를 만들기
train_dataset = SFT_dataset(data_path_1_SFT=data_path_1_SFT, tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [16]:
# 디코딩 확인
def remove_ignore_idx(token_ids, ignore_idx = -100):
    return [token_id for token_id in token_ids if token_id != ignore_idx]

print(tokenizer.decode(remove_ignore_idx(train_dataset.input_ids[0])))
print(tokenizer.decode(remove_ignore_idx(train_dataset.labels[0])))

### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>
'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>


In [17]:
torch.cuda.empty_cache()

In [18]:
training_args = TrainingArguments(
    output_dir="/aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16=True
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [19]:
trainer.train()
model.save_pretrained('/aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.984100
1000,2.776800
1500,2.687200


In [20]:
# generator
generator = pipeline('text-generation', model='/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 불고기용 고기의 종류와 양에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기는 쇠고기와 함께 먹는 음식 중 하나입니다. 따라서 불고기를 먹을 수 있는 종류는 다양합니다. 예를 들어, 닭가슴살 스테이크, 오므라이스 샐러드 등이 있습니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 42대 부통령직을 수행했습니다.作)作)은 "리처드 닉슨"이 41대 부통령을 수행한 년도를 가리키는 말입니다.作)는 "리처드 닉슨"이 40대 부통령을 맡았던 년도를 의미합니다.作은 "리처드슨"이 50대 부통령

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다.子供共和國際空港)이라고 불립니다.子供公和国際空港이라는 뜻입니다.子供空和國際公港이라는 이름을 가진 항공사는 다음과 같습니다.\n\n1. 대한항공

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇으로써 미세먼지 정보를 알 수 없습니다. 미세먼지 예보를 확인해 보시는 것이 좋겠습니다.\n\n미세먼지 예보: 일반적으로 미세먼지는 주로 중국에서 발원하여 중국 전역으로 퍼져나가기 때문에 중국발 미세먼지가 유입될


In [21]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# NLTK 데이터 다운로드
nltk.download('punkt')

# 정답
references = [
    "한우는 불고기용 고기로 많이 사용됩니다.",
    "리처드 닉슨은 1953년부터 1961년까지 미국의 부통령을 지냈습니다.",
    "시카고 오헤어 국제공항은 미국 일리노이주 시카고에 위치해 있습니다.",
    "오늘 미세먼지 정보는 날씨 앱이나 웹사이트를 통해 확인할 수 있습니다."
]

# KoGPT-2 모델 답변
kogpt2_responses = [
    "불고기용 고기 한우에요?! 육질이 부드럽고, 쫀득하고 부드러운 육회가 정말 맛있었어요!! 매콤한 소고기향이 나면서 입에서 살살 녹는 맛이 일품인 것 같네요.ᄒᄒ",
    "리처드 닉슨이 43대 부통령직을 수행한 년도는?이다. 그럼에도 불구하고 부시 대통령은 자신의 임기 중 재임기간 동안 가장 눈부신 업적을 거둔 대통령으로 기억되고 있다. 1961년 10월 9일의 선거에서 민주당이 하원 다수당을 차지하게 되고 공화당은 상원",
    "시카고 오헤어 국제공항은 어디에 있어? 알았어. 호텔에 있을 때 잠깐 봤잖아. 아, 그러세요. 아뇨. 잘 모르겠어요. 그녀의 얼굴에는 미소까지 지으며 그녀가 웃고 있었다.",
    "오늘 미세먼지 어때? 그래서 오늘 날씨도 좋아서 공기 질이 좀 좋았겠네요. 아~ 공기 질 좋았던 거죠? 네. 오늘 날씨는 어땠는지요. 예. 그니까요. 어~ 아까 말씀드"
]

# SFT 모델 답변
sft_responses = [
    "### Instruction(명령어):\n불고기용 고기 한우에요?\n\n### Response(응답):'저는 인공지능 어시스턴트이기 때문에 불고기용 고기의 종류와 양에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기는 쇠고기와 함께 먹는 음식 중 하나입니다. 따라서 불고기를 먹을 수 있는 종류는 다양합니다. 예를 들어, 닭가슴살 스테이크, 오므라이스 샐러드 등이 있습니다.",
    "### Instruction(명령어):\n리처드 닉슨이 43대 부통령직을 수행한 년도는?\n\n### Response(응답):'리처드 닉슨은 42대 부통령직을 수행했습니다.作)은 '리처드 닉슨'이 41대 부통령을 수행한 년도를 가리키는 말입니다.作는 '리처드 닉슨'이 40대 부통령을 맡았던 년도를 의미합니다.作은 '리처드슨'이 50대 부통령",
    "### Instruction(명령어):\n시카고 오헤어 국제공항은 어디에 있어?\n\n### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다.子供共和國際空港)이라고 불립니다.子供公和国際空港이라는 뜻입니다.子供空和國際公港이라는 이름을 가진 항공사는 다음과 같습니다.\n\n1. 대한항공",
    "### Instruction(명령어):\n오늘 미세먼지 어때?\n\n### Response(응답):'저는 인공지능 챗봇으로써 미세먼지 정보를 알 수 없습니다. 미세먼지 예보를 확인해 보시는 것이 좋겠습니다.\n\n미세먼지 예보: 일반적으로 미세먼지는 주로 중국에서 발원하여 중국 전역으로 퍼져나가기 때문에 중국발 미세먼지가 유입될"
]

# BLEU 점수 계산
def calculate_bleu(references, responses):
    bleu_scores = []
    for ref, res in zip(references, responses):
        bleu_scores.append(sentence_bleu([ref.split()], res.split(), smoothing_function=SmoothingFunction().method1))
    return bleu_scores

# BLEU 점수
kogpt2_bleu_scores = calculate_bleu(references, kogpt2_responses)
sft_bleu_scores = calculate_bleu(references, sft_responses)

print("KoGPT-2 BLEU 점수:", kogpt2_bleu_scores)
print("SFT 모델 BLEU 점수:", sft_bleu_scores)


[nltk_data] Downloading package punkt to /aiffel/nltk_data...


KoGPT-2 BLEU 점수: [0.009629943614188135, 0.005648892151960285, 0.04099093841051792, 0.016020720994064933]
SFT 모델 BLEU 점수: [0.004286566854494811, 0.007678432706586176, 0.04521782889827669, 0.012813098990837714]


[nltk_data]   Package punkt is already up-to-date!


KoGPT-2 모델은 자연스럽지 못 하고 내용이 이어지지 못 함. SFT 모델은 문장이 좀 더 자연스럽고, 정확성 면에서도 개선되었으나 여전히 부정확.  
SFT 모델의 BLEU 점수가 KoGPT-2 모델보다 높음. SFT 모델이 KoGPT-2 모델보다 전반적으로 더 나은 성능을 보임.

In [22]:
torch.cuda.empty_cache()

## Reward Model

In [23]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [24]:
# reward model
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [26]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# ranking dataset 만들기
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [28]:
# pairwiseloss
class PairWiseLoss(nn.Module):

    def forward(self, chosen_reward: torch.Tensor, reject_reward: torch.Tensor) -> torch.Tensor:
        probs = torch.sigmoid(chosen_reward - reject_reward)
        log_probs = torch.log(probs)
        loss = -log_probs.mean()
        return loss

In [29]:
# 훈련 데이터셋 만들기
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [30]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 937.60it/s]


In [31]:
# 데이터 확인
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [32]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [33]:
trainer.fit(use_lora=0)

model.save_pretrained('aiffel/KoChatGPT/output_2_RM')

Train epoch: 100%|██████████| 1/1 [03:58<00:00, 238.25s/it]


In [34]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -1.3


In [35]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -1.0


In [36]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: -0.8


In [37]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -0.6


In [38]:
torch.cuda.empty_cache()

## Proximal Policy Optimization
PPO에 사용할 actor모델은 1단계 SFT 모델을, critic모델은 2단계 RM 모델을 사용.

In [39]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [40]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [41]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [42]:
# model
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [43]:
# 데이터 import 토크나이징
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [44]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [45]:
len(list_prompt)

12000

In [46]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [47]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:18<00:00,  6.12s/it]


In [48]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이므로 현재 상황을 알지 못합니다. 하지만 일반적으로 불고기는 일본 홋카이도현의 전통 음식으로, 일본의 유명한 지역 특산물 중 하나입니다. 따라서 불고기는 직접 확인해 보시는 것은 추천합니다. 챗봇으로 참여하며, 일반적으로 식당 메뉴와 가격은 불고기를 포함한 다른 음식에도 적용되는 경우가 많습니다. 하지만 고기를 먹을 경우 건강에 위험을 초래할 수도 있으니, 음식점에서는 해당 음식을 권장하고 선택하시기를 권장합니다. 따르면, 불고기는 불고기용으로 많이 사용됩니다. 챗봇으로 참여하며, 불고기는 일본 식품 중 하나이며, 음식점에서는 불고기를 선택하면 불고기를 제공합니다. 챗봇은 실제로 불고기용을 제공하는 경우가 없는 경우도 있으므로 상황에 따라 선택해야 합니다. 챗봇은 일본의 문화 전통으로 알려져 있으며, 불고기를 선택하거나 불고기의 상태를 조절해 먹는 것은 가능합니다. 챗봇은 일본의 전통적인 음식점에 해당하는 음식점 중 하나이며, 불고기의 상태를 조절하며, 불고기를 선택하는 것은 일본 식당의 특징 중 하나입니다. 챗봇은 불고기를 선택한 후

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'저는 데이터에 대한 의존도를 가지고 있지 않기 때문에 정확한 답변을 제공할 수 없습니다. 하지만 톰슨(Jamson)은 32대에 부통령직을 수행한 것으로 알려져 있습니다. 닉슨이 부통령직을 수행한 일 년도는 1970년입니다. 문법상으로는 "리처드 닉슨"이 32대 부통령직을 수행하였습니다. コイクト ンクンシンサンイクサイクーーンシーンシンスックシンーンシンシンターーキンシンシー キンセシンー キンーンーーキンーーーー キンジーーキンシンシー キンスキンシー ーキンナーン キンジンシンナーンジンジンシンシンター 

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어

### Response(

In [49]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# 기준 텍스트 (레퍼런스)
reference_responses = [
    ["불고기용", "고기는", "한우입니다."],
    ["리처드", "닉슨은", "43대", "부통령직을", "수행한", "년도는", "1953년입니다."],
    ["시카고", "오헤어", "국제공항은", "시카고에", "있습니다."],
    ["오늘", "미세먼지는", "나쁩니다."]
]

# SFT 모델 생성 텍스트
sft_generated_responses = [
    "저는 인공지능 어시스턴트이기 때문에 불고기용 고기의 종류와 양에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기는 쇠고기와 함께 먹는 음식 중 하나입니다. 따라서 불고기를 먹을 수 있는 종류는 다양합니다. 예를 들어, 닭가슴살 스테이크, 오므라이스 샐러드 등이 있습니다.",
    "리처드 닉슨은 42대 부통령직을 수행했습니다.作)作)은 '리처드 닉슨'이 41대 부통령을 수행한 년도를 가리키는 말입니다.作)는 '리처드 닉슨'이 40대 부통령을 맡았던 년도를 의미합니다.作은 '리처드슨'이 50대 부통령",
    "시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다.子供共和國際空港)이라고 불립니다.子供公和国際空港이라는 뜻입니다.子供空和國際公港이라는 이름을 가진 항공사는 다음과 같습니다.\n\n1. 대한항공",
    "저는 인공지능 챗봇으로써 미세먼지 정보를 알 수 없습니다. 미세먼지 예보를 확인해 보시는 것이 좋겠습니다.\n\n미세먼지 예보: 일반적으로 미세먼지는 주로 중국에서 발원하여 중국 전역으로 퍼져나가기 때문에 중국발 미세먼지가 유입될"
]

# PPO 모델 생성 텍스트
ppo_generated_responses = [
    "저는 인공지능 챗봇이므로 제가 어떤 종류의 쇠고기를 판매하는지 알 수 없습니다. 죄송합니다. srkin (쇠고기) 추가 정보가 필요합니다. 불고기용 한우는 불고기용 부위를 말합니다. geos, 또는 kinhungsin에서 각각 다른 이름입니다. geos, 또는 geos, 혹은 kinhungsin에서는 다양한 이름으로 사용됩니다. geos, 또는  Kinhungsin은 각각 다른 이름으로 사용되기도 합니다.  ninglawaid에서 각각 다른 이름으로 사용될 수 있습니다. geos, 또는  Ninglawaid에서는 주로 불고기용으로 사용되는 경우가 많습니다. geos, 또는 geos는 각각 다른 이름으로 사용될 수 있습니다. geos, 또는 geos는 각각 다른 이름으로 쓰일 수 있습니다. geos, 또는 geos는 각각 다른 이름으로 사용되기도 합니다. neos, 또는",
    "저는 데이터에 대한 의존도를 가지고 있지 않기 때문에 제임스 닉슨의 부통령직에서 근무한 기간도에 대해 알지 못합니다. 정확한 정보와 관련해서는 '리처드 닉슨의 47대 부통령직'이라는 개인의 이름이나 연도, 인물 정보에 대한 정보가 없습니다., 'token': 147}自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由自由",
    "저는 인공지능 챗봇이므로 시카고에 있는 국제공항 정보나 위치 정보가 없습니다. 하지만 현재 시카고에는 수많은 공항이 있으므로 이 정보를 토대로 정보를 얻으실 수 있습니다.神明神話題: '시카고 오헤어 국제공항은 어디에 있어?' 究算神話題: '시카고에 위치한 국제공항은 어디로 가고 있나?'라는 문맥에서 언급됩니다. 究算神話題: '시카고에 위치한 국제공항은?'라는 문맥에서 언급됩니다.神話題: '시카고에 위치한 국제공항은 어디로 가고 있는지에 따라?'라는 문맥에서 언급됩니다.神話題: '이 국제공항은이런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런저런",
    "미세먼지 때문에 걱정된다면 대중교통, 생활 밀접물, 음식물류 등의 분야로 대처하는 것이 좋습니다. 또한, 미세먼지를 줄이기 위해 실내에서 미세먼지를 제거하는 것도 도움이 될 수 있습니다. 또한, 미세먼지 발생 시 마스크 착용 등의 예방책과 손세탁 등으로 미세먼지를 줄이는 방법을 통해 미세먼지가 줄어들기 쉽습니다.\n\n또한, 미세먼지는 우리가 건강하지 않은 환경에서는 오히려 불쾌한 환경이기 때문에 주의가 필요합니다.\n\n마스크는 미세먼지 제거가 중요하지만, 미세먼지 발생 시 마스크 착용은 필수적인 것이며, 마스크를 착용하지 않으면 건강을 잃는 것이 불가피합니다. 身間なな間間, all\n\n그러나 이런 상황에서는 대중교통, 생활밀착물, 음식물류 등에 대한 예방책이 필요합니다. all\n마스크 착용, 손세탁 등 다양한 방법으로 미세먼지를 줄이는 것이 도움이 됩니다. all\n하지만 실제로 미세먼지라는 상황에 대처하기 위해서는 각 산업분야의 기술 개발과 관리체계가 잘 확립"
]

# BLEU 점수 계산
smoothing_function = SmoothingFunction().method1

for reference, sft_response, ppo_response in zip(reference_responses, sft_generated_responses, ppo_generated_responses):
    sft_bleu = sentence_bleu([reference], sft_response.split(), smoothing_function=smoothing_function)
    ppo_bleu = sentence_bleu([reference], ppo_response.split(), smoothing_function=smoothing_function)
    print(f"SFT BLEU: {sft_bleu:.4f}, PPO BLEU: {ppo_bleu:.4f}")


SFT BLEU: 0.0050, PPO BLEU: 0.0021
SFT BLEU: 0.0208, PPO BLEU: 0.0000
SFT BLEU: 0.0128, PPO BLEU: 0.0079
SFT BLEU: 0.0067, PPO BLEU: 0.0017


SFT의 BLEU score가 전체적으로 높게 나옴. 중복된 토큰이 등장하지 않아서 그런 것같음.  
SFT 모델은 주어진 질문에 대한 응답이 논리적이며 일관성을 가지고 있지만, 정확한 정보 제공에서는 한계가 있음. PPO 모델은 강화 학습을 통해 추가 조정된 모델로, 일부 응답에서 불필요한 반복이 발생하지만, 유창성 면에서는 비교적 양호.

In [64]:
torch.cuda.empty_cache()

## 데이터셋 추가
[KorQuAD 2.0](https://korquad.github.io/)

In [51]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

In [52]:
from datasets import load_dataset

# Load the KorQuAD 2.0 dataset
dataset = load_dataset('leeseeun/KorQuAD_2.0')

  0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 83486
    })
})
{'question': '드라마 예고범의 감독은 누구일까?', 'answer': '나카무라 요시히로, 히라바야시 카츠토시, 사와다 메구미', 'text': '## 질문: 드라마 예고범의 감독은 누구일까?\n## 답변: 나카무라 요시히로, 히라바야시 카츠토시, 사와다 메구미\n\n'}


In [54]:
import json
import os
import random
from datasets import load_dataset
import transformers
from torch.utils.data import Dataset
from dataclasses import dataclass
import torch
import copy

# 1. Load and Transform KorQuAD 2.0 Dataset
dataset = load_dataset('leeseeun/KorQuAD_2.0')

korquad_data = []
for item in dataset['train']:
    korquad_data.append({
        'prompt': item['question'],
        'completion': item['answer']
    })

# 2. Load the original SFT Dataset
data_path_1_SFT = '~/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'
data_path_1_SFT = os.path.expanduser(data_path_1_SFT)

with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    sft_data = json.load(json_file)

# 3. Combine both datasets
combined_data = sft_data + korquad_data

# 4. Shuffle the combined dataset
random.shuffle(combined_data)
combined_data = combined_data[:20000]

# 5. Save the combined dataset
output_path = '~/aiffel/KoChatGPT/data_kochatgpt/combined_data.jsonl'
output_path = os.path.expanduser(output_path)

with open(output_path, "w", encoding='utf-8-sig') as json_file:
    json.dump(combined_data, json_file, ensure_ascii=False, indent=4)

  0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [56]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [57]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [58]:
train_dataset = SFT_dataset(data_path_1_SFT='./aiffel/KoChatGPT/data_kochatgpt/combined_data.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375, 10559,   397,  9363, 14604, 18309, 14237, 11754,
        24796, 19850, 11181, 11554, 10604,   375,   378,   378,   378, 41951,
          454,  9549, 20549,   383,  8142,  7192, 14914, 48718,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100, 48718,     1])


In [59]:
# train_dataset.input_ids[0]를 디코딩해보세요.
def remove_ignore_idx(token_ids, ignore_idx = -100):
    return [token_id for token_id in token_ids if token_id != ignore_idx]

print(tokenizer.decode(remove_ignore_idx(train_dataset.input_ids[0])))
print(tokenizer.decode(remove_ignore_idx(train_dataset.labels[0])))

### Instruction(명령어):
1964년 올림픽 축구 남자부에서 우승을 거둔 참가국은?

### Response(응답):헝가리</s>
헝가리</s>


In [65]:
training_args = TrainingArguments(
    output_dir="/aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=5,
    prediction_loss_only=True,
    gradient_accumulation_steps=8,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [66]:
trainer.train()
model.save_pretrained('/aiffel/KoChatGPT/output_1_SFT')

Step,Training Loss
500,3.797700


In [67]:
generator = pipeline('text-generation', model='/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 일반적으로 불고기용 쇠고기는 돼지고기와 함께 먹는 경우가 많습니다. 따라서 고기를 먹을 수 있는 음식점은 직접 방문하여 확인해보시는 것이 좋습니다.) : "불고기용 쇠고기 한우에요?", 'token':

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):1987년)高速)苦速)貯速)告速)膏速)考速)故高速)高速]高速 (고속)誥速)高速[고속]

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오헤이어 국제공항은 미국 일리노이주 시카고에 위치해 있습니다.岳, Mountain, Michael, Soul, Taylor, Hollywood, Freeze 등이 위치하고 있습니다.岳 : 시카고에서 가장 아름다운 도시 중 하나입니다.岳 : 시카

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이므로, 미세먼지 관련 정보를 알 수 없습니다. 저는 인공지능 어시스턴트이기 때문에 정확한 답변을 드리기 어렵습니다. 하지만 미세먼지 예보나 기상정보 등을 참고하시는 것을 추천드립니다.) : "미세먼지 예보는


12000개의 데이터로 학습했을 때보다 텍스트의 일관성이 있음.